In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Load data from google drive -------------------------------------------------
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'https://drive.google.com/file/d/1-cUe1Kx5kqawmbKxkqXfGVHT6LB3UvaI/view?usp=drive_link'

import pandas as pd

# to get the id part of the file
id = link.split("/")[-2]

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('equity_chars_ret.csv')

df = pd.read_csv('equity_chars_ret.csv')

<ipython-input-1-5a49f3eaacba>:26: DtypeWarning: Columns (98) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('equity_chars_ret.csv')


In [3]:
df.head()

,Unnamed: 0,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,...,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,RET
0,0,10001,20000131,20825.0000,0.062780,0.003941,0.093899,8.271964,0.025807,0.078753,...,NaN,0.012581,4.016382e-06,0.050882,0.022777,1.078260,0.597444,3.818182e+00,49.0,-0.044118
1,1,10002,20000131,94197.1875,0.487762,0.237912,0.026658,8.875427,0.050570,-0.036174,...,6.0,0.035375,3.703263e-06,0.070352,0.028031,1.163444,0.172986,7.636364e+00,60.0,-0.025641
2,2,10009,20000131,34028.2500,0.299177,0.089507,-0.093446,10.366105,0.035277,-0.036174,...,3.0,0.031896,2.535039e-07,0.017241,0.015675,1.046077,4.829668,2.881086e-08,60.0,-0.008475
3,3,10012,20000131,154794.3125,2.389731,5.710816,2.007230,13.700713,0.145356,0.914538,...,3.0,0.112558,1.265005e-08,0.236364,0.078982,0.712583,23.904388,3.353590e-09,36.0,-0.097276
4,4,10016,20000131,337277.3750,0.619844,0.384206,-0.183942,11.928143,0.039299,0.325924,...,2.0,0.043094,8.944278e-08,0.056604,0.028425,1.005188,2.595967,3.859758e-08,38.0,-0.099338


In [15]:
#Drop null values to be able to run MLP
df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
df.dropna()

print(len(df))
print(df['ms'])


1665729
0          NaN
1          6.0
2          3.0
3          3.0
4          2.0
          ... 
1665724    3.0
1665725    6.0
1665726    6.0
1665727    2.0
1665728    6.0
Name: ms, Length: 1665729, dtype: float64


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm

In [9]:
#Split Data
seed=42
features = df.loc[:,df.columns!='RET'].to_numpy()
target = df['RET'].to_numpy()

features = torch.tensor(features, dtype=torch.float32)
target = torch.tensor(target, dtype=torch.float32)

train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.2, random_state=seed)
train_features, val_features, train_target, val_target = train_test_split(train_features, train_target, test_size=0.2, random_state=seed)

scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
train_features = torch.tensor(train_features, dtype=torch.float32)
#df_X_val = scaler.transform(df_X_val)
#df_X_test = scaler.transform(df_X_test)


In [10]:
#Create DataLoaders
train_dataset = TensorDataset(train_features, train_target)
train_loader = DataLoader(train_dataset, batch_size=10000, shuffle=True)

val_dataset = TensorDataset(val_features, val_target)
val_loader = DataLoader(val_dataset, batch_size=10000, shuffle=True)

test_dataset = TensorDataset(test_features, test_target)
test_loader = DataLoader(test_dataset, batch_size=10000, shuffle=True)

[MLP for Regression Tutorial](https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-create-a-neural-network-for-regression-with-pytorch.md)

[MLP for regression tutorial 2](https://www.kaggle.com/code/kanji10494/simple-mlp-regressor-using-pytorch)

In [11]:
#MLP Model
#Authors use ReLU as activaiton function and 3 hidden layers with 32,16, and 8 neurons respectively.
class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(98, 32),
      nn.ReLU(),
      nn.Linear(32, 16),
      nn.ReLU(),
      nn.Linear(16, 1)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

In [12]:
#Initialize the MLP
model = MLP()

#Define the loss function and optimizer
loss_function = nn.L1Loss()   #This is MAE loss.
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# Run the training loop
for epoch in range(0, 5): # 5 epochs at maximum

  # Print epoch
  print(f'Starting epoch {epoch+1}')

  # Set current loss value
  current_loss = 0.0

  # Iterate over the DataLoader for training data
  for i, data in enumerate(train_loader, 0):

    # Get and prepare inputs
    inputs, targets = data
    inputs, targets = inputs.float(), targets.float()
    targets = targets.reshape((targets.shape[0], 1))

    # Zero the gradients
    optimizer.zero_grad()

    # Perform forward pass
    outputs = model(inputs)

    # Compute loss
    loss = loss_function(outputs, targets)

    # Perform backward pass
    loss.backward()

    # Perform optimization
    optimizer.step()

    # Print statistics
    current_loss += loss.item()
    if i % 10 == 0:
        print('Loss after mini-batch %5d: %.3f' %
              (i + 1, current_loss / 500))
        current_loss = 0.0

# Process is complete.
print('Training process has finished.')